In [1]:
import pandas as pd
import io, requests, os 
from bs4 import BeautifulSoup
from tqdm import tqdm
pd.set_option('display.max_columns', None)

start, end = "2020-03-01", "2022-12-31"
output_folder = 'raw_dataset2'
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# Cases
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-03-01. Note that data from today can take 3 days to update in this file.

## Download

In [2]:
response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv")

file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object)
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,

## Preprocess

In [3]:
# drop and rename columns
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

# filter non-counties
df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df.fillna(0, inplace=True)

# only select cases within date range
selected_date_columns = [col for col in df.columns if (col!='FIPS' and col>=start and col <= end)]
df = df[['FIPS'] + selected_date_columns]

## Dump

In [9]:
df.to_csv(f'{output_folder}/Cases.csv', index=False)

# Vaccination
From [CDC vaccinations in the US counties](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh). Data collection starts from 2020-12-13. Note that data from today can take upto 3 days to update in the CDC server.

In [10]:
response = requests.get("https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD")
file_object = io.StringIO(response.content.decode('utf-8'))

df = pd.read_csv(file_object, low_memory=False)
# df = pd.read_csv("COVID-19_Vaccinations_in_the_United_States_County.csv", low_memory=False)
df.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,Series_Complete_Yes,Series_Complete_Pop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Series_Complete_5to17,Series_Complete_5to17Pop_Pct,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Booster_Doses,Booster_Doses_Vax_Pct,Booster_Doses_5Plus,Booster_Doses_5Plus_Vax_Pct,Booster_Doses_12Plus,Booster_Doses_12Plus_Vax_Pct,Booster_Doses_18Plus,Booster_Doses_18Plus_Vax_Pct,Booster_Doses_50Plus,Booster_Doses_50Plus_Vax_Pct,Booster_Doses_65Plus,Booster_Doses_65Plus_Vax_Pct,Second_Booster_50Plus,Second_Booster_50Plus_Vax_Pct,Second_Booster_65Plus,Second_Booster_65Plus_Vax_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_5PlusPop_Pct_SVI,Series_Complete_5to17Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Metro_status,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_5PlusPop_Pct_UR_Equity,Series_Complete_5to17Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity,Booster_Doses_Vax_Pct_SVI,Booster_Doses_12PlusVax_Pct_SVI,Booster_Doses_18PlusVax_Pct_SVI,Booster_Doses_65PlusVax_Pct_SVI,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop,Bivalent_Booster_5Plus,Bivalent_Booster_5Plus_Pop_Pct,Bivalent_Booster_12Plus,Bivalent_Booster_12Plus_Pop_Pct,Bivalent_Booster_18Plus,Bivalent_Booster_18Plus_Pop_Pct,Bivalent_Booster_65Plus,Bivalent_Booster_65Plus_Pop_Pct
0,05/10/2023,55129,19,Washburn County,WI,96.7,11123.0,70.8,11097.0,73.9,10863.0,78.5,10368.0,81.3,4749.0,95.0,10325.0,65.7,10311.0,68.7,648.0,28.8,10105.0,73.0,9663.0,75.7,4483.0,95.0,6800.0,65.9,6800.0,65.9,6754.0,66.8,6599.0,68.3,5558.0,75.6,3649.0,81.4,3538.0,63.7,2635.0,72.2,A,3.0,3.0,1.0,3.0,3.0,4.0,Non-metro,7.0,7.0,5.0,7.0,7.0,8.0,4.0,4.0,4.0,4.0,8.0,8.0,8.0,8.0,15720.0,15010.0,2252.0,13847.0,12758.0,4304.0,3978.0,26.5,3955.0,28.6,3899.0,30.6,2588.0,60.1
1,05/10/2023,19173,19,Taylor County,IA,97.3,3149.0,51.4,3145.0,55.0,3079.0,59.8,2966.0,63.3,1249.0,89.7,2951.0,48.2,2949.0,51.5,157.0,15.2,2887.0,56.0,2792.0,59.6,1211.0,87.0,1804.0,61.1,1804.0,61.2,1796.0,62.2,1774.0,63.5,1447.0,74.3,974.0,80.4,864.0,59.7,668.0,68.6,A,1.0,2.0,1.0,2.0,2.0,4.0,Non-metro,5.0,6.0,5.0,6.0,6.0,8.0,4.0,4.0,4.0,4.0,8.0,8.0,8.0,8.0,6121.0,5721.0,1034.0,5152.0,4687.0,1392.0,841.0,14.7,841.0,16.3,834.0,17.8,588.0,42.2
2,05/10/2023,36059,19,Nassau County,NY,97.5,1391226.0,95.0,1384503.0,95.0,1329779.0,95.0,1232671.0,95.0,313054.0,95.0,1179481.0,86.9,1174924.0,91.6,133132.0,61.4,1127179.0,95.0,1041792.0,95.0,268167.0,95.0,548555.0,46.5,548367.0,46.7,540617.0,48.0,514670.0,49.4,319930.0,57.7,168258.0,62.7,140803.0,44.0,88821.0,52.8,B,8.0,8.0,8.0,8.0,8.0,8.0,Metro,4.0,4.0,4.0,4.0,4.0,4.0,7.0,7.0,7.0,6.0,3.0,3.0,3.0,2.0,1356924.0,1282825.0,216857.0,1169773.0,1065968.0,246690.0,236202.0,18.4,231748.0,19.8,224931.0,21.1,109947.0,44.6
3,05/10/2023,48281,19,Lampasas County,TX,98.9,11678.0,54.5,11660.0,57.4,11468.0,61.9,10849.0,64.6,3920.0,90.7,10007.0,46.7,9999.0,49.2,669.0,19.0,9845.0,53.2,9330.0,55.5,3448.0,79.8,4217.0,42.1,4217.0,42.2,4206.0,42.7,4144.0,44.4,3392.0,54.8,2187.0,63.4,1582.0,46.6,1151.0,52.6,C,9.0,9.0,9.0,10.0,10.0,11.0,Metro,1.0,1.0,1.0,2.0,2.0,3.0,11.0,11.0,11.0,10.0,3.0,3.0,3.0,2.0,21428.0,20320.0,3521.0,18515.0,16799.0,4322.0,1951.0

In [11]:
df = df[(df['FIPS']!='UNK') & (~df['FIPS'].isna())]
# df.dropna(axis=0, inplace=True)
df = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']]

df['Date'] = pd.to_datetime(df['Date'])
df = df[
    (df['Date']>=pd.to_datetime(start)) & 
    (df['Date']<=pd.to_datetime(end))
]
# df = df.sort_values(by=['FIPS', 'Date']).reset_index(drop=True)
df.fillna(0, inplace=True)

In [12]:
df = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']].pivot_table(
    values='Series_Complete_Pop_Pct',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)
df.head()

,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00,2020-12-25 00:00:00,2020-12-26 00:00:00,2020-12-27 00:00:00,2020-12-28 00:00:00,2020-12-29 00:00:00,2020-12-30 00:00:00,2020-12-31 00:00:00,2021-01-01 00:00:00,2021-01-02 00:00:00,2021-01-03 00:00:00,2021-01-04 00:00:00,2021-01-05 00:00:00,2021-01-06 00:00:00,2021-01-07 00:00:00,2021-01-08 00:00:00,2021-01-09 00:00:00,2021-01-10 00:00:00,2021-01-11 00:00:00,2021-01-12 00:00:00,2021-01-13 00:00:00,2021-01-14 00:00:00,2021-01-15 00:00:00,2021-01-16 00:00:00,2021-01-17 00:00:00,2021-01-18 00:00:00,2021-01-19 00:00:00,2021-01-20 00:00:00,2021-01-21 00:00:00,2021-01-22 00:00:00,2021-01-23 00:00:00,2021-01-24 00:00:00,2021-01-25 00:00:00,2021-01-26 00:00:00,2021-01-27 00:00:00,2021-01-28 00:00:00,2021-01-29 00:00:00,2021-01-30 00:00:00,2021-01-31 00:00:00,2021-02-01 00:00:00,2021-02-02 00:00:00,2021-02-03 00:00:00,2021-02-04 00:00:00,2021-02-05 00:00:00,2021-02-06 00:00:00,2021-02-07 00:00:00,2021-02-08 00:00:00,2021-02-09 00:00:00,2021-02-10 00:00:00,2021-02-11 00:00:00,2021-02-12 00:00:00,2021-02-13 00:00:00,2021-02-14 00:00:00,2021-02-15 00:00:00,2021-02-16 00:00:00,2021-02-17 00:00:00,2021-02-18 00:00:00,2021-02-19 00:00:00,2021-02-20 00:00:00,2021-02-21 00:00:00,2021-02-22 00:00:00,2021-02-23 00:00:00,2021-02-24 00:00:00,2021-02-25 00:00:00,2021-02-26 00:00:00,2021-02-27 00:00:00,2021-02-28 00:00:00,2021-03-01 00:00:00,2021-03-02 00:00:00,2021-03-03 00:00:00,2021-03-04 00:00:00,2021-03-05 00:00:00,2021-03-06 00:00:00,2021-03-07 00:00:00,2021-03-08 00:00:00,2021-03-09 00:00:00,2021-03-10 00:00:00,2021-03-11 00:00:00,2021-03-12 00:00:00,2021-03-13 00:00:00,2021-03-14 00:00:00,2021-03-15 00:00:00,2021-03-16 00:00:00,2021-03-17 00:00:00,2021-03-18 00:00:00,2021-03-19 00:00:00,2021-03-20 00:00:00,2021-03-21 00:00:00,2021-03-22 00:00:00,2021-03-23 00:00:00,2021-03-24 00:00:00,2021-03-25 00:00:00,2021-03-26 00:00:00,2021-03-27 00:00:00,2021-03-28 00:00:00,2021-03-29 00:00:00,2021-03-30 00:00:00,2021-03-31 00:00:00,2021-04-01 00:00:00,2021-04-02 00:00:00,2021-04-03 00:00:00,2021-04-04 00:00:00,2021-04-05 00:00:00,2021-04-06 00:00:00,2021-04-07 00:00:00,2021-04-08 00:00:00,2021-04-09 00:00:00,2021-04-10 00:00:00,2021-04-11 00:00:00,2021-04-12 00:00:00,2021-04-13 00:00:00,2021-04-14 00:00:00,2021-04-15 00:00:00,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00,2021-04-26 00:00:00,2021-04-27 00:00:00,2021-04-28 00:00:00,2021-04-29 00:00:00,2021-04-30 00:00:00,2021-05-01 00:00:00,2021-05-02 00:00:00,2021-05-03 00:00:00,2021-05-04 00:00:00,2021-05-05 00:00:00,2021-05-06 00:00:00,2021-05-07 00:00:00,2021-05-08 00:00:00,2021-05-09 00:00:00,2021-05-10 00:00:00,2021-05-11 00:00:00,2021-05-12 00:00:00,2021-05-13 00:00:00,2021-05-14 00:00:00,2021-05-15 00:00:00,2021-05-16 00:00:00,2021-05-17 00:00:00,2021-05-18 00:00:00,2021-05-19 00:00:00,2021-05-20 00:00:00,2021-05-21 00:00:00,2021-05-22 00:00:00,2021-05-23 00:00:00,2021-05-24 00:00:00,2021-05-25 00:00:00,2021-05-26 00:00:00,2021-05-27 00:00:00,2021-05-28 00:00:00,2021-05-29 00:00:00,2021-05-30 00:00:00,2021-05-31 00:00:00,2021-06-01 00:00:00,2021-06-02 00:00:00,2021-06-03 00:00:00,2021-06-04 00:00:00,2021-06-05 00:00:00,2021-06-06 00:00:00,2021-06-07 00:00:00,2021-06-08 00:00:00,2021-06-09 00:00:00,2021-06-10 00:00:00,2021-06-11 00:00:00,2021-06-12 00:00:00,2021-06-13 00:00:00,2021-06-14 00:00:00,2021-06-15 00:00:00,2021-06-16 00:00:00,2021-06-17 00:00:00,2021-06-18 00:00:00,2021-06-19 00:00:00,2021-06-20 00:00:00,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00,2021-06-25 00:00:00,2021-06-26 00:00:00,2021-06-27 00:00:00,2021-06-28 00:00:00,2021-06-29 00:00:00,2021-06-30 00:

In [16]:
def fill_missing_dates(data):
    df = data.copy()
    df = df.fillna(method='ffill').fillna(0)
    # this assumes the dates are along the column axis
    date_columns = sorted([pd.to_datetime(col) for col in df.columns if col!='FIPS'])
    non_date_columns = [col for col in df.columns if col not in date_columns]
    df.columns = non_date_columns + date_columns

    missing_dates = []
    previous_date = date_columns[0]
    
    for date in date_columns[1:]:
        steps = (date - previous_date).days
        if steps > 1:
            for step in range(1, steps):
                missing_dates.append(previous_date+pd.to_timedelta(step, unit='D'))
        previous_date = date

    print(f'Number of missing dates {len(missing_dates)}')

    df = df.reindex(columns = ['FIPS'] + sorted(date_columns + missing_dates))
    df = df.T.fillna(method='ffill').T
    return df

In [17]:
# this was added because after May 2022, 
# vaccination data from CDC started having random missing dates
df = fill_missing_dates(df)
df.head(2)

Number of missing dates 167


,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00,2020-12-25 00:00:00,2020-12-26 00:00:00,2020-12-27 00:00:00,2020-12-28 00:00:00,2020-12-29 00:00:00,2020-12-30 00:00:00,2020-12-31 00:00:00,2021-01-01 00:00:00,2021-01-02 00:00:00,2021-01-03 00:00:00,2021-01-04 00:00:00,2021-01-05 00:00:00,2021-01-06 00:00:00,2021-01-07 00:00:00,2021-01-08 00:00:00,2021-01-09 00:00:00,2021-01-10 00:00:00,2021-01-11 00:00:00,2021-01-12 00:00:00,2021-01-13 00:00:00,2021-01-14 00:00:00,2021-01-15 00:00:00,2021-01-16 00:00:00,2021-01-17 00:00:00,2021-01-18 00:00:00,2021-01-19 00:00:00,2021-01-20 00:00:00,2021-01-21 00:00:00,2021-01-22 00:00:00,2021-01-23 00:00:00,2021-01-24 00:00:00,2021-01-25 00:00:00,2021-01-26 00:00:00,2021-01-27 00:00:00,2021-01-28 00:00:00,2021-01-29 00:00:00,2021-01-30 00:00:00,2021-01-31 00:00:00,2021-02-01 00:00:00,2021-02-02 00:00:00,2021-02-03 00:00:00,2021-02-04 00:00:00,2021-02-05 00:00:00,2021-02-06 00:00:00,2021-02-07 00:00:00,2021-02-08 00:00:00,2021-02-09 00:00:00,2021-02-10 00:00:00,2021-02-11 00:00:00,2021-02-12 00:00:00,2021-02-13 00:00:00,2021-02-14 00:00:00,2021-02-15 00:00:00,2021-02-16 00:00:00,2021-02-17 00:00:00,2021-02-18 00:00:00,2021-02-19 00:00:00,2021-02-20 00:00:00,2021-02-21 00:00:00,2021-02-22 00:00:00,2021-02-23 00:00:00,2021-02-24 00:00:00,2021-02-25 00:00:00,2021-02-26 00:00:00,2021-02-27 00:00:00,2021-02-28 00:00:00,2021-03-01 00:00:00,2021-03-02 00:00:00,2021-03-03 00:00:00,2021-03-04 00:00:00,2021-03-05 00:00:00,2021-03-06 00:00:00,2021-03-07 00:00:00,2021-03-08 00:00:00,2021-03-09 00:00:00,2021-03-10 00:00:00,2021-03-11 00:00:00,2021-03-12 00:00:00,2021-03-13 00:00:00,2021-03-14 00:00:00,2021-03-15 00:00:00,2021-03-16 00:00:00,2021-03-17 00:00:00,2021-03-18 00:00:00,2021-03-19 00:00:00,2021-03-20 00:00:00,2021-03-21 00:00:00,2021-03-22 00:00:00,2021-03-23 00:00:00,2021-03-24 00:00:00,2021-03-25 00:00:00,2021-03-26 00:00:00,2021-03-27 00:00:00,2021-03-28 00:00:00,2021-03-29 00:00:00,2021-03-30 00:00:00,2021-03-31 00:00:00,2021-04-01 00:00:00,2021-04-02 00:00:00,2021-04-03 00:00:00,2021-04-04 00:00:00,2021-04-05 00:00:00,2021-04-06 00:00:00,2021-04-07 00:00:00,2021-04-08 00:00:00,2021-04-09 00:00:00,2021-04-10 00:00:00,2021-04-11 00:00:00,2021-04-12 00:00:00,2021-04-13 00:00:00,2021-04-14 00:00:00,2021-04-15 00:00:00,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00,2021-04-26 00:00:00,2021-04-27 00:00:00,2021-04-28 00:00:00,2021-04-29 00:00:00,2021-04-30 00:00:00,2021-05-01 00:00:00,2021-05-02 00:00:00,2021-05-03 00:00:00,2021-05-04 00:00:00,2021-05-05 00:00:00,2021-05-06 00:00:00,2021-05-07 00:00:00,2021-05-08 00:00:00,2021-05-09 00:00:00,2021-05-10 00:00:00,2021-05-11 00:00:00,2021-05-12 00:00:00,2021-05-13 00:00:00,2021-05-14 00:00:00,2021-05-15 00:00:00,2021-05-16 00:00:00,2021-05-17 00:00:00,2021-05-18 00:00:00,2021-05-19 00:00:00,2021-05-20 00:00:00,2021-05-21 00:00:00,2021-05-22 00:00:00,2021-05-23 00:00:00,2021-05-24 00:00:00,2021-05-25 00:00:00,2021-05-26 00:00:00,2021-05-27 00:00:00,2021-05-28 00:00:00,2021-05-29 00:00:00,2021-05-30 00:00:00,2021-05-31 00:00:00,2021-06-01 00:00:00,2021-06-02 00:00:00,2021-06-03 00:00:00,2021-06-04 00:00:00,2021-06-05 00:00:00,2021-06-06 00:00:00,2021-06-07 00:00:00,2021-06-08 00:00:00,2021-06-09 00:00:00,2021-06-10 00:00:00,2021-06-11 00:00:00,2021-06-12 00:00:00,2021-06-13 00:00:00,2021-06-14 00:00:00,2021-06-15 00:00:00,2021-06-16 00:00:00,2021-06-17 00:00:00,2021-06-18 00:00:00,2021-06-19 00:00:00,2021-06-20 00:00:00,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00,2021-06-25 00:00:00,2021-06-26 00:00:00,2021-06-27 00:00:00,2021-06-28 00:00:00,2021-06-29 00:00:00,2021-06-30 00:

In [19]:
df['FIPS'] = df['FIPS'].astype(int)
df.to_csv(f'{output_folder}/Vaccination.csv', index=False)

# Weekly cases by age groups

Data [source](https://data.cdc.gov/Public-Health-Surveillance/COVID-19-Weekly-Cases-and-Deaths-by-Age-Race-Ethni/hrdz-jaxc). This table summarizes COVID-19 case and death data submitted to CDC as case reports for the line-level dataset. Case and death counts are stratified according to sex, age, and race and ethnicity at regional and national levels.

Weekly cumulative counts with five or fewer cases or deaths are not reported to protect confidentiality of patients. Records with unknown or missing sex, age, or race and ethnicity and of multiple, non-Hispanic race and ethnicity are included in case and death totals. 

## Download

In [2]:
download_link = "https://data.cdc.gov/api/views/hrdz-jaxc/rows.csv?accessType=DOWNLOAD"
response = requests.get(download_link)
file_object = io.StringIO(response.content.decode('utf-8'))

df = pd.read_csv(file_object)
df.head()

,end_of_week,jurisdiction,age_group,sex,race_ethnicity_combined,case_count_suppressed,death_count_suppressed,case_crude_rate_suppressed_per_100k,death_crude_rate_suppressed_per_100k
0,2022-06-11,Region 7,30 - 39 Years,Female,"AI/AN, NH",NaN,NaN,NaN,NaN
1,2023-03-18,Region 4,40 - 49 Years,Male,"Black, NH",112.0,NaN,13.89,NaN
2,2020-05-30,Region 7,30 - 39 Years,Female,"AI/AN, NH",8.0,NaN,158.98,NaN
3,2021-03-13,Region 4,40 - 49 Years,Male,"Black, NH",728.0,7.0,90.32,0.87
4,2020-12-26,Region 4,40 - 49 Years,Male,"Black, NH",1230.0,10.0,152.59,1.24


## Proprocess

In [4]:
# need to count only at national level
df = df[
    (df['jurisdiction']=='US') & 
    (df['age_group']!='Overall') & \
    (df['sex']=='Overall') &\
    (df['race_ethnicity_combined']=='Overall')    
]
df.drop(columns=['jurisdiction', 'sex', 'race_ethnicity_combined'], axis=1, inplace=True)

In [5]:
df = df[(df['end_of_week']>=start) & (df['end_of_week']<=end)]

In [7]:
df.rename(
    {'case_count_suppressed':'cases'}, 
    axis=1, inplace=True
)

In [9]:
# aggregate over all races and sex groups
df = df.groupby(
    ['end_of_week', 'age_group']
)[['cases']].aggregate('sum').reset_index()

In [10]:
# pivot the age groups with weekly covid cases
df = df.pivot(
    index='end_of_week',
    columns='age_group',
    values='cases'
).reset_index()
df.head(3)

age_group,end_of_week,0 - 4 Years,12 - 15 Years,16 - 17 Years,18 - 29 Years,30 - 39 Years,40 - 49 Years,5 - 11 Years,50 - 64 Years,65 - 74 Years,75+ Years
0,2020-03-07,39.0,50.0,48.0,751.0,1042.0,1278.0,62.0,2466.0,1172.0,860.0
1,2020-03-14,139.0,169.0,144.0,4366.0,5171.0,5650.0,169.0,9187.0,4066.0,2654.0
2,2020-03-21,419.0,614.0,475.0,15867.0,18433.0,19053.0,538.0,30614.0,12454.0,8153.0


In [11]:
df['AGE517'] = df[
    ['5 - 11 Years','12 - 15 Years', '16 - 17 Years']
].sum(axis=1)

df.drop(
    columns=['5 - 11 Years','12 - 15 Years', '16 - 17 Years'], 
    inplace=True
)

In [12]:
age_group_mapping = {
    '0 - 4 Years':'UNDER5', 'AGE517': 'AGE517',
    '18 - 29 Years':'AGE1829', '30 - 39 Years': 'AGE3039',
    '40 - 49 Years':'AGE4049', '50 - 64 Years': 'AGE5064',
    '65 - 74 Years': 'AGE6574', '75+ Years': 'AGE75PLUS'
}
df.rename(age_group_mapping, axis=1, inplace=True)
age_groups = list(age_group_mapping.values())

df['Overall'] = df[age_groups].sum(axis=1)

In [13]:
df.to_csv(f'{output_folder}/Cases by age groups.csv', index=False)